In [357]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import r2_score

In [358]:
games = pd.read_csv('../input/nba-games/games.csv')
details = pd.read_csv('../input/nba-games/games_details.csv')
teams = pd.read_csv('../input/nba-games/teams.csv')
players = pd.read_csv('../input/nba-games/players.csv')
ranking = pd.read_csv('../input/nba-games/ranking.csv')

C:\Users\Clay\AppData\Local\Temp\ipykernel_97368\977498853.py:2: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  details = pd.read_csv('../input/nba-games/games_details.csv')


In [359]:
def get_labels(ranking):
    temp = ranking.copy(deep=False)
    temp = temp.groupby(['TEAM_ID','SEASON_ID'])['G','W'].max()
    temp = pd.DataFrame(temp)
    temp.reset_index(inplace=True)
    drops = []
    for i in range(len(temp)):
        if temp.iloc[i,1] / 10000 > 2:
            temp.iloc[i,1] = temp.iloc[i,1] % 10000
        else:
            drops.append(i)
            continue;
        if (temp.iloc[i,2] != 82):
            drops.append(i)
    for i in range(len(drops)):
        temp.drop([drops[i]], inplace=True)
    temp.reset_index(inplace=True)
    temp.drop(columns=['index'], inplace=True)
    temp.drop(columns=['G'], inplace=True)
#     temp = pd.merge(temp, ranking, how='left', left_on=['TEAM_ID','STANDINGSDATE'], right_on = ['TEAM_ID','STANDINGSDATE'])
#     temp.drop(columns=['STANDINGSDATE','LEAGUE_ID','SEASON_ID_y','CONFERENCE','TEAM','G','W','L','HOME_RECORD','ROAD_RECORD','RETURNTOPLAY'], inplace=True)
    return temp

In [360]:
labels = get_labels(ranking)
labels

C:\Users\Clay\AppData\Local\Temp\ipykernel_97368\989031293.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  temp = temp.groupby(['TEAM_ID','SEASON_ID'])['G','W'].max()


,TEAM_ID,SEASON_ID,W
0,1610612737,2002,35
1,1610612737,2003,28
2,1610612737,2004,13
3,1610612737,2005,26
4,1610612737,2006,30
...,...,...,...
501,1610612766,2015,48
502,1610612766,2016,36
503,1610612766,2017,36
504,1610612766,2018,39


In [361]:
def get_features(games, details):
    temp = pd.merge(games, details, how='left', left_on=['GAME_ID'], right_on = ['GAME_ID'])
    temp = temp[['TEAM_ID','SEASON','FGM','FGA','FG_PCT','FG3M','FG3A','FG3_PCT','FTM','FTA'
             ,'FT_PCT','OREB','DREB','REB','AST','STL','BLK','TO','PF','PTS','PLUS_MINUS']]
    temp = temp.groupby(['TEAM_ID','SEASON'])['FGM','FGA','FG_PCT','FG3M','FG3A','FG3_PCT','FTM','FTA'
             ,'FT_PCT','OREB','DREB','REB','AST','STL','BLK','TO','PF','PTS','PLUS_MINUS'].sum()
    temp = pd.DataFrame(temp)
    next_season = []
    temp.reset_index(inplace=True)
    for i in range(len(temp)):
        next_season.append(temp.iloc[i,1] + 1)
    temp['NEXT_SEASON'] = next_season
    return temp

In [362]:
features = get_features(games, details)
features

C:\Users\Clay\AppData\Local\Temp\ipykernel_97368\2543770804.py:5: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  temp = temp.groupby(['TEAM_ID','SEASON'])['FGM','FGA','FG_PCT','FG3M','FG3A','FG3_PCT','FTM','FTA'


,TEAM_ID,SEASON,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,...,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS,NEXT_SEASON
0,1.610613e+09,2003,2857.0,6609.0,320.764,422.0,1256.0,104.083,1555.0,2004.0,...,2543.0,3548.0,1666.0,633.0,410.0,1292.0,1853.0,7691.0,-1905.0,2004
1,1.610613e+09,2004,2997.0,6821.0,357.705,309.0,997.0,114.160,1456.0,2059.0,...,2387.0,3510.0,1649.0,642.0,350.0,1289.0,2068.0,7759.0,-3983.0,2005
2,1.610613e+09,2005,3196.0,6997.0,358.522,450.0,1205.0,138.741,1804.0,2404.0,...,2427.0,3568.0,1759.0,633.0,417.0,1377.0,2235.0,8646.0,-1950.0,2006
3,1.610613e+09,2006,3105.0,6986.0,369.110,385.0,1161.0,127.166,1865.0,2438.0,...,2528.0,3595.0,1728.0,663.0,480.0,1400.0,2195.0,8460.0,-1950.0,2007
4,1.610613e+09,2007,3457.0,7668.0,382.692,436.0,1250.0,122.879,2129.0,2758.0,...,2885.0,4074.0,2084.0,696.0,536.0,1411.0,2016.0,9479.0,-1165.0,2008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
594,1.610613e+09,2018,3495.0,7788.0,413.804,1043.0,2949.0,245.989,1619.0,2046.0,...,2980.0,3824.0,2029.0,625.0,435.0,1038.0,1679.0,9652.0,-330.0,2019
595,1.610613e+09,2019,2615.0,6015.0,297.664,839.0,2397.0,180.742,1135.0,1506.0,...,2249.0,3012.0,1668.0,466.0,297.0,1008.0,1333.0,7204.0,-2260.0,2020
596,1.610613e+09,2020,3210.0,7117.0,352.732,1095.0,2976.0,211.146,1299.0,1729.0,...,2704.0,3559.0,2173.0,640.0,386.0,1160.0,1485.0,8814.0,-899.0,2021
597,1.610613e+09,2021,3685.0,7945.0,398.470,1190.0,3301.0,218.024,1373.0,1859.0,...,2934.0,3877.0,2409.0,747.0,428.0,1129.0,1736.0,9933.0,-380.0,2022


In [363]:
def swap_columns(df, col1, col2):
    col_list = list(df.columns)
    x, y = col_list.index(col1), col_list.index(col2)
    col_list[y], col_list[x] = col_list[x], col_list[y]
    df = df[col_list]
    return df

In [364]:
def get_data(ranking, games, details):
    labels = get_labels(ranking)
    features = get_features(games, details)
    data = pd.merge(labels, features, how='left', left_on=['TEAM_ID','SEASON_ID'], right_on = ['TEAM_ID','NEXT_SEASON'])
    data.drop(columns=['SEASON_ID','SEASON'], inplace=True)
    data.dropna(inplace=True)
    data = swap_columns(data, 'W', 'NEXT_SEASON')
    data = data.astype({'NEXT_SEASON': 'int64'})
    data.rename(columns={'W' : 'NEXT_W'}, inplace=True)
    data.reset_index(inplace=True)
    data.drop(columns=['index'], inplace=True)
    return data

In [365]:
data = get_data(ranking, games, details)
data_magic = data.loc[(data['TEAM_ID'] == 1610612753) & (data['NEXT_SEASON'] <= 2018)]
data_magic

C:\Users\Clay\AppData\Local\Temp\ipykernel_97368\989031293.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  temp = temp.groupby(['TEAM_ID','SEASON_ID'])['G','W'].max()
C:\Users\Clay\AppData\Local\Temp\ipykernel_97368\2543770804.py:5: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  temp = temp.groupby(['TEAM_ID','SEASON'])['FGM','FGA','FG_PCT','FG3M','FG3A','FG3_PCT','FTM','FTA'


,TEAM_ID,NEXT_SEASON,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,...,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS,NEXT_W
239,1610612753,2004,2967.0,6906.0,335.257,435.0,1269.0,117.705,1514.0,2049.0,...,2400.0,3419.0,1615.0,568.0,317.0,1097.0,1791.0,7883.0,-2880.0,36
240,1610612753,2005,3100.0,6828.0,360.657,333.0,945.0,101.382,1819.0,2394.0,...,2608.0,3666.0,1621.0,654.0,454.0,1304.0,1989.0,8351.0,-926.0,36
241,1610612753,2006,3102.0,6601.0,387.765,320.0,873.0,107.207,1808.0,2492.0,...,2560.0,3523.0,1600.0,593.0,393.0,1296.0,2083.0,8332.0,-440.0,40
242,1610612753,2007,3292.0,6976.0,431.547,389.0,1092.0,138.565,1944.0,2767.0,...,2763.0,3817.0,1754.0,645.0,462.0,1509.0,2220.0,8917.0,140.0,52
243,1610612753,2008,3635.0,7653.0,427.772,939.0,2432.0,231.376,1936.0,2683.0,...,3196.0,4129.0,2008.0,604.0,415.0,1382.0,2016.0,10145.0,2320.0,59
244,1610612753,2009,3968.0,8681.0,471.271,1065.0,2836.0,264.234,2235.0,3103.0,...,3641.0,4723.0,2164.0,783.0,597.0,1510.0,2330.0,11236.0,2965.0,59
245,1610612753,2010,3756.0,7977.0,474.926,1066.0,2827.0,251.127,2077.0,2888.0,...,3445.0,4464.0,2017.0,644.0,592.0,1429.0,2134.0,10655.0,3675.0,52
246,1610612753,2012,2496.0,5706.0,286.447,731.0,1966.0,165.446,1100.0,1663.0,...,2258.0,3063.0,1452.0,494.0,303.0,1033.0,1316.0,6823.0,-10.0,20
247,1610612753,2013,3404.0,7575.0,388.699,550.0,1664.0,145.450,1140.0,1513.0,...,2873.0,3847.0,2063.0,581.0,397.0,1280.0,1778.0,8498.0,-2865.0,23
248,1610612753,2014,3318.0,7450.0,389.891,615.0,1759.0,186.870,1430.0,1885.0,...,2904.0,3807.0,1879.0,696.0,405.0,1326.0,1877.0,8681.0,-2230.0,25


In [366]:
x = data_magic[data_magic.columns.difference(['NEXT_W'])]
y = data_magic[['NEXT_W']]

In [367]:
y

,NEXT_W
239,36
240,36
241,40
242,52
243,59
244,59
245,52
246,20
247,23
248,25


In [368]:
scaler_x = StandardScaler()
scaler_y = StandardScaler()
x_scaled = scaler_x.fit_transform(x)
y_scaled = scaler_y.fit_transform(y)

In [369]:
X_train = x_scaled[:13]
X_test = x_scaled[13]
y_train = y_scaled[:13]
y_test = y_scaled[13]


In [370]:
X_train, X_test, y_train, y_test = train_test_split(x_scaled, y_scaled, test_size=1, shuffle=False)

In [371]:
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
seq_len = X_train[0].shape[0]

In [372]:
print(y_test)

[[0.30791675]]


In [373]:
class NBA(Dataset):
    def __init__(self, X_train, y_train):
        super(Dataset, self).__init__()
        self.X_train = X_train
        self.y_train = y_train

    def __len__(self):
        return len(self.X_train)
    
    def __getitem__(self, idx):
        x = self.X_train[idx]
        y = self.y_train[idx]
        return x, y

In [374]:
hidden_size = 2048
num_layers = 1
learning_rate = 0.0005
batch_size = 32
epoch_size = 10

In [375]:
train_dataset = NBA(X_train, y_train)
test_dataset = NBA(X_test, y_test)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [376]:
class RNN(nn.Module):
    def __init__(self, input_feature_size, hidden_size, num_layers):
        super(RNN, self).__init__()
        self.lstm = nn.LSTM(input_feature_size, hidden_size, num_layers, batch_first=True)
        self.fc1 = nn.Linear(hidden_size, 128)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(128, 1)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [377]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
rnn = RNN(21, hidden_size, num_layers).to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(rnn.parameters(), lr=0.0001)

In [378]:
rnn.train()
for epoch in range(epoch_size):
    loss = 0.0

    for batch_idx, data in enumerate(train_loader):
        inputs, targets = data
        inputs.to(device)
        targets.to(device)

        optimizer.zero_grad()

        outputs = rnn(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        loss += loss.item()
        if batch_idx % 100 == 99:
            print(f'[{epoch + 1}, {batch_idx + 1:5d}] loss: {loss / 100:.3f}')
            loss = 0.0

In [379]:
prediction = []
ground_truth = []

rnn.eval()
with torch.no_grad():
    for data in test_loader:
        inputs, targets = data
        inputs = inputs.to(device)

        ground_truth += targets.flatten().tolist()
        out = rnn(inputs).detach().cpu().flatten().tolist()
        prediction += out

In [380]:
prediction_2d = np.array([prediction])
ground_truth_2d = np.array([ground_truth])
prediction

[0.031066469848155975]

In [381]:
prediction = scaler_y.inverse_transform(prediction_2d)
ground_truth = scaler_y.inverse_transform(ground_truth_2d)

In [382]:
r2score = r2_score(prediction, ground_truth)
print(r2score)

nan


C:\Users\Clay\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


In [383]:
for i in range(len(prediction)):
    print("Prediction: ", prediction[i], ", ground truth: ", ground_truth[i])
    print(prediction[i] - ground_truth[i])

Prediction:  [38.4677917] , ground truth:  [42.]
[-3.5322083]
